# 0. Install and Import Dependencies

In [14]:
!pip install mediapipe opencv-python pandas scikit-learn

In [15]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [16]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# 1. Make Some Detections

In [17]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        #print(results.right_hand_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
        

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord(' '):
            break

cap.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV
<!--<img src="https://i.imgur.com/8bForKY.png">-->
<!--<img src="https://i.imgur.com/AzKNp7A.png">-->

In [19]:
import csv
import os
import numpy as np

In [20]:
num_coords = 21 #Hand has 21 landmarks

In [21]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [22]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [126]:
class_name = "Y" # From A to Z to add the landmarks to the csv file

In [128]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks / useless at this point
        #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
         #                        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
          #                       mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
           #                      )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections / useless at this point
        #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
        #                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
        #                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        #                         )
        # Export coordinates
       # try:
            # Extract Left and Rigth hand landmarks / one at the time
        if results.right_hand_landmarks:
            pose = results.right_hand_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            row = pose_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
                
           # except:
            #    print('no')
             #   pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord(' '):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [129]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [130]:
df = pd.read_csv('coords.csv')

In [131]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z19,v19,x20,y20,z20,v20,x21,y21,z21,v21
0,A,0.872361,1.014420,-6.802763e-07,0.0,0.802850,0.984900,-0.013200,0.0,0.742364,...,-0.032420,0.0,0.876929,0.847015,-0.018758,0.0,0.878708,0.875686,-0.003740,0.0
1,A,0.880794,1.000834,-5.970348e-07,0.0,0.808890,0.980843,-0.013410,0.0,0.745570,...,-0.025347,0.0,0.875507,0.838928,-0.011486,0.0,0.881289,0.864441,0.002510,0.0
2,A,0.875478,1.000710,-6.209229e-07,0.0,0.812720,0.981782,-0.017534,0.0,0.750220,...,-0.039444,0.0,0.887454,0.849501,-0.027056,0.0,0.888212,0.878121,-0.012948,0.0
3,A,0.877295,0.996915,-5.995648e-07,0.0,0.810931,0.981167,-0.017814,0.0,0.747848,...,-0.035215,0.0,0.885406,0.846887,-0.022345,0.0,0.888111,0.872026,-0.008568,0.0
4,A,0.870285,0.995014,-6.071867e-07,0.0,0.805751,0.976099,-0.014567,0.0,0.745917,...,-0.033790,0.0,0.879430,0.849899,-0.021579,0.0,0.882398,0.876090,-0.007873,0.0


In [132]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z19,v19,x20,y20,z20,v20,x21,y21,z21,v21
4854,Y,0.301277,0.713866,-4.999960e-07,0.0,0.376766,0.687557,-0.021483,0.0,0.443821,...,-0.037228,0.0,0.274309,0.341676,-0.039328,0.0,0.276596,0.289744,-0.035333,0.0
4855,Y,0.300141,0.712820,-4.968692e-07,0.0,0.376357,0.686942,-0.020894,0.0,0.444729,...,-0.035632,0.0,0.274595,0.343193,-0.039521,0.0,0.276802,0.291937,-0.037399,0.0
4856,Y,0.300001,0.712719,-4.860838e-07,0.0,0.375955,0.687031,-0.021030,0.0,0.444419,...,-0.035067,0.0,0.274257,0.343622,-0.039129,0.0,0.276493,0.291918,-0.037097,0.0
4857,Y,0.300780,0.711728,-4.852828e-07,0.0,0.375800,0.686995,-0.020841,0.0,0.444320,...,-0.034069,0.0,0.273890,0.341194,-0.037652,0.0,0.276375,0.288789,-0.035182,0.0
4858,Y,0.301566,0.713592,-4.809959e-07,0.0,0.376861,0.686566,-0.019396,0.0,0.444327,...,-0.033639,0.0,0.274778,0.342040,-0.036795,0.0,0.277258,0.290253,-0.034068,0.0


In [133]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [135]:
y_test

3757    T
2783    O
2181    L
2697    O
2963    P
       ..
4346    W
2124    L
495     C
2231    L
3066    P
Name: class, Length: 1458, dtype: object

## 3.2 Train Machine Learning Classification Model

In [136]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [137]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [138]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\Users\Algor Lombako\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [139]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [59]:
fit_models['rc'].predict(X_test)

array(['B', 'C', 'F', 'F', 'A', 'C', 'F', 'B', 'A', 'D', 'F', 'D', 'C',
       'A', 'B', 'D', 'D', 'H', 'D', 'E', 'D', 'D', 'E', 'C', 'H', 'A',
       'H', 'E', 'F', 'B', 'D', 'D', 'D', 'D', 'E', 'F', 'B', 'D', 'B',
       'C', 'H', 'C', 'A', 'A', 'D', 'C', 'A', 'D', 'A', 'E', 'G', 'D',
       'H', 'A', 'H', 'C', 'G', 'G', 'A', 'B', 'D', 'B', 'D', 'A', 'G',
       'A', 'H', 'D', 'A', 'G', 'C', 'E', 'G', 'A', 'C', 'A', 'D', 'D',
       'G', 'D', 'E', 'E', 'A', 'D', 'H', 'F', 'C', 'D', 'D', 'E', 'A',
       'A', 'C', 'F', 'G', 'D', 'H', 'H', 'B', 'H', 'C', 'A', 'F', 'F',
       'H', 'E', 'A', 'B', 'B', 'B', 'D', 'F', 'B', 'C', 'C', 'C', 'F',
       'D', 'F', 'G', 'F', 'A', 'D', 'B', 'G', 'A', 'D', 'E', 'A', 'D',
       'D', 'G', 'F', 'D', 'A', 'F', 'F', 'G', 'G', 'F', 'H', 'G', 'H',
       'B', 'G', 'B', 'B', 'H', 'G', 'B', 'C', 'C', 'D', 'F', 'A', 'D',
       'C', 'A', 'C', 'F', 'F', 'B', 'G', 'D', 'H', 'H', 'B', 'D', 'D',
       'A', 'A', 'F', 'A', 'A', 'C', 'A', 'E', 'H', 'G', 'A', 'C

## 3.3 Evaluate and Serialize Model 

In [140]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [141]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9890260631001372
rc 0.9554183813443072
rf 0.9945130315500685
gb 0.9821673525377229


In [142]:
fit_models['rf'].predict(X_test)

array(['T', 'O', 'L', ..., 'C', 'L', 'P'], dtype=object)

In [143]:
y_test

3757    T
2783    O
2181    L
2697    O
2963    P
       ..
4346    W
2124    L
495     C
2231    L
3066    P
Name: class, Length: 1458, dtype: object

In [144]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [145]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [146]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [1]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
         #                        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
          #                       mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                #                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
         #                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
          #                       mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
           #                      )

        if results.right_hand_landmarks:
            pose = results.right_hand_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            row = pose_row

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord(' '):
            break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [ ]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))